# TransLuxPop — VIIRS & WorldPop Automation (Original Logic, Organized)
This is a **structure-only cleanup** of the original notebook. ✅

**Preserved (unchanged):**
- Your original code cells (same code, same imports, same functions)
- Your original dependencies / library usage
- Your original computation logic and parameter choices

**Changed (safe refactor):**
- Added English section headers
- Grouped cells for readability (imports → config → utilities → extraction → batch run → post-processing)
- No semantic modifications were made to your algorithms

> Goal: maximum reproducibility with better readability.


## 0. Imports & Environment

In [1]:
import pandas as pd
import autofill
import numpy as np
import rasterio
from tqdm import tqdm
from math import cos,radians

## 3. Load Grid Table / Initialize DataFrame

In [3]:

filename = "grids_set_3.xlsx"

df = pd.read_excel(filename)
float(df['lon_max'][0]), 
len_df = len(df)
len_df

10000

In [4]:
df.head(3)

,grid_id,year,nation_code,lat_min,lon_min,lat_max,lon_max,cell_area,region_type,city_type,...,covid_intensity,VIIRS_last_year,WorldPop_last_year,VIIRS,WorldPop,dVIIRS,dlogVIIRS,dWorldPop,City,description
0,1,2015,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
1,1,2016,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
2,1,2017,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."


## 1. Configuration (Paths, Years, Parameters)

In [5]:
for i in tqdm(range(min(len_df, len(df)))):
    lat_min = df["lat_min"][i]
    lon_min = df["lon_min"][i]
    lat_max = df["lat_max"][i]
    lon_max = df["lon_max"][i]
    year = df["year"][i]

    df.loc[df.index[i], "VIIRS"] = float(autofill.getavgviirs(lon_min, lat_min, lon_max, lat_max, year))
    df.loc[df.index[i], "WorldPop"] = float(autofill.getavgworldpop(lon_min, lat_min, lon_max, lat_max, year))

    #print("GridID: "+str(i)+" Finished Calculating")

100%|██████████| 10000/10000 [09:59<00:00, 16.67it/s]


In [6]:
df.head(5)

,grid_id,year,nation_code,lat_min,lon_min,lat_max,lon_max,cell_area,region_type,city_type,...,covid_intensity,VIIRS_last_year,WorldPop_last_year,VIIRS,WorldPop,dVIIRS,dlogVIIRS,dWorldPop,City,description
0,1,2015,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,31.911856,9096.751953,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
1,1,2016,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,34.980362,9196.083984,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
2,1,2017,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,33.471649,9292.364258,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
3,1,2018,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,36.807701,9383.561523,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
4,1,2019,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,38.425270,9474.592773,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."


## 4. Batch Extraction Loop (with Checkpoints)

In [8]:
df.to_excel(filename + "_processed.xlsx", index=False)


In [9]:
df[["VIIRS","WorldPop"]].isna().sum()


VIIRS       0
WorldPop    0
dtype: int64

In [10]:
df[["VIIRS","WorldPop"]].describe()


,VIIRS,WorldPop
count,10000.000000,10000.000000
mean,49.461696,7897.302629
std,35.592737,6751.094220
min,0.000000,-9999.000000
25%,24.957561,2886.773743
50%,42.647346,6444.637451
75%,64.543303,11614.969238
max,253.627304,38132.140625


## 这里坐一下Cell Area的计算

In [24]:
for i in tqdm.tqdm(range(min(len_df, len(df)))):
    lat_min = df["lat_min"][i]
    lon_min = df["lon_min"][i]
    lat_max = df["lat_max"][i]
    lon_max = df["lon_max"][i]

    dlon = lon_max-lon_min
    dlat = lat_max-lat_min
    lat = (lat_max+lat_min)/2
    adlon = dlon*111.32*cos(radians(lat))
    adlat = dlat*111.32

    df.loc[df.index[i], "cell_area"] = adlon*adlat
    

100%|██████████| 10000/10000 [00:02<00:00, 4063.58it/s]


# 基础数据处理

In [17]:
import pandas as pd
import numpy as np
import tqdm
filename = "grids_set_3.xlsx_processed.xlsx"

In [18]:
df = pd.read_excel(filename)
df

,grid_id,year,nation_code,lat_min,lon_min,lat_max,lon_max,cell_area,region_type,city_type,...,covid_intensity,VIIRS_last_year,WorldPop_last_year,VIIRS,WorldPop,dVIIRS,dlogVIIRS,dWorldPop,City,description
0,1,2015,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,31.911856,9096.751953,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
1,1,2016,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,34.980362,9196.083984,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
2,1,2017,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,33.471649,9292.364258,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
3,1,2018,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,36.807701,9383.561523,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
4,1,2019,IDN,3.554544,98.679221,3.569844,98.694521,NaN,industrial,rural,...,NaN,NaN,NaN,38.425270,9474.592773,NaN,NaN,NaN,NaN,"Industrial zone in Medan, chosen for rural urb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1000,2020,DEU,53.524233,10.004952,53.535774,10.016493,NaN,industrial,major,...,NaN,NaN,NaN,32.640247,1110.477783,NaN,NaN,NaN,NaN,"Industrial zone in Hamburg, chosen for interna..."
9996,1000,2021,DEU,53.524233,10.004952,53.535774,10.016493,NaN,industrial,major,...,NaN,NaN,NaN,34.868061,1116.137695,NaN,NaN,NaN,NaN,"Industrial zone in Hamburg, chosen for interna..."
9997,1000,2022,DEU,53.524233,10.004952,53.535774,10.016493,NaN,industrial,major,...,NaN,NaN,NaN,31.280800,1124.326782,NaN,NaN,NaN,NaN,"Industrial zone in Hamburg, chosen for interna..."
9998,1000,2023,DEU,53.524233,10.004952,53.535774,10.016493,NaN,industrial,major,...,NaN,NaN,NaN,33.578987,1141.128296,NaN,NaN,NaN,NaN,"Industrial zone in Hamburg, chosen for interna..."


In [19]:

for i in tqdm.tqdm(range(len_df)):

    if i%10 == 0:
        df.loc[df.index[i], "VIIRS_last_year "] = np.nan
        df.loc[df.index[i], "WorldPop_last_year"] = np.nan
        df.loc[df.index[i], "dVIIRS"] = np.nan
        df.loc[df.index[i], "dWorldPop"] = np.nan
        df.loc[df.index[i], "dlogVIIRS"] = np.nan
        continue

    df.loc[df.index[i], "VIIRS_last_year "] = df["VIIRS"][i-1]
    df.loc[df.index[i], "WorldPop_last_year"] = df["WorldPop"][i-1]
    df.loc[df.index[i], "dVIIRS"] = df["VIIRS"][i]-df["VIIRS_last_year "][i]
    df.loc[df.index[i], "dWorldPop"] = df["WorldPop"][i]-df["WorldPop_last_year"][i]
    df.loc[df.index[i], "dlogVIIRS"] = np.log1p(df["VIIRS"][i])-np.log1p(df["VIIRS_last_year "][i])


100%|██████████| 10000/10000 [00:08<00:00, 1150.79it/s]


In [20]:
print(df.index[:15])
print(df.index[-15:])
print("index 连续吗:", df.index.equals(pd.RangeIndex(len(df))))


RangeIndex(start=0, stop=15, step=1)
RangeIndex(start=9985, stop=10000, step=1)
index 连续吗: True


In [25]:
df.to_excel("grids_set_3_processed.xlsx", index=False)